In [ ]:
# --- Section 1: Setup and Data Preparation ---

# Mount Google Drive to access your files
from google.colab import drive
drive.mount('/content/drive')

# Install the Ultralytics library for YOLO models
!pip install ultralytics

# Path to your zip file in Google Drive
# Make sure this path is correct based on where you uploaded the zip file
zip_file_path = '/content/drive/MyDrive/DOCUMENTS/export-#Vwb9qfJIcnarc8Txefn8 (2).zip'

# Destination folder where the data will be unzipped
destination_path = '/content/my_project_data'

# Unzip the file
!unzip {zip_file_path} -d {destination_path}

# --- Section 2: Update data.yaml File Paths ---

# This step is crucial to prevent FileNotFoundError and ensure the training script finds the correct paths
import os
import yaml

data_yaml_path = os.path.join(destination_path, 'data.yaml')

# Read the data.yaml file
with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

# Update the paths to be absolute paths for Colab
data_yaml['path'] = destination_path
data_yaml['train'] = 'images'
data_yaml['val'] = 'images'

# Write the updated dictionary back to the data.yaml file
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print("data.yaml updated successfully.")

# --- Section 3: Model Training ---

from ultralytics import YOLO

# Path to your updated data.yaml file
data_yaml_path = os.path.join(destination_path, 'data.yaml')

# Load a pre-trained YOLOv8 segmentation model
model = YOLO('yolov8n-seg.pt')

# Train the model with your custom dataset for 100 epochs
# The model will save its training results and weights
print("Starting model training...")
results = model.train(
    data=data_yaml_path,
    epochs=100
)

# --- Section 4: Video Tracking Demo (Optional, after model training is complete) ---

print("\nOnce training is complete, you can use the code below for video tracking.")
print("Remember to replace the placeholder paths with your trained model and video file.")

# from ultralytics import YOLO
# import cv2
# import json
# import os

# model_path = '/content/runs/segment/train/weights/best.pt'
# video_path = '/content/drive/MyDrive/path/to/your/video.mp4'
# output_json_path = 'results.json'

# model = YOLO(model_path)
# cap = cv2.VideoCapture(video_path)
# tracking_results = []
# frame_count = 0

# while cap.isOpened():
#     success, frame = cap.read()
#     if not success:
#         break
#     frame_count += 1
#     results = model.track(frame, persist=True, tracker="bytetrack.yaml")
#     if results[0].boxes.id is not None:
#         boxes = results[0].boxes
#         for box_id, class_id in zip(boxes.id, boxes.cls):
#             object_id = int(box_id.cpu().numpy())
#             class_name = model.names[int(class_id.cpu().numpy())]
#             bbox = boxes.xyxy[0].cpu().numpy().astype(int).tolist()
#             tracking_results.append({
#                 "frame": frame_count,
#                 "id": object_id,
#                 "class": class_name,
#                 "bounding_box": bbox
#             })
# cap.release()
# cv2.destroyAllWindows()

# with open(output_json_path, 'w') as f:
#     json.dump(tracking_results, f, indent=4)

# print(f"Tracking results exported to {output_json_path}")